In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

## Single Agent

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3)
search.invoke("Who are the top stars of the Eurocup 2024?")

[{'url': 'https://www.givemesport.com/ranking-the-15-best-players-at-euro-2024/',
  'content': 'Best 11 At Euro 2024 Boasting the likes of Lamine Yamal and Cody Gakpo, there have been plenty of stars strutting their stuff in Germany.'},
 {'url': 'https://www.90min.com/features/10-best-players-euro-2024-ranked',
  'content': "The 10 best players of Euro 2024 - ranked The 10 best players of Euro 2024 - ranked The best players of Euro 2024 The Three Lions number one was once again in superb form for his nation throughout Euro 2024, particularly in the early rounds when England's attackers were struggling for form at the other end of the pitch. There was point during Euro 2024 in which Jamal Musiala looked certain to win Player of the Tournament. As expected, he was bloody brilliant at Euro 2024 for Spain and picked up the Player of the Tournament award, but he's not our winner..."},
 {'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',


In [8]:
tools = [search]

In [9]:
tools

[TavilySearchResults(max_results=3)]

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [12]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='9f957b45-5e00-4a2c-907b-93df0c59c174'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gx8w', 'function': {'arguments': '{"query":"Where is the soccer Eurocup 2024 played?"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 950, 'total_tokens': 1007, 'completion_time': 0.162857143, 'prompt_time': 0.047943294, 'queue_time': 0.050765007, 'total_time': 0.210800437}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-097322d3-17c8-4620-8fe3-cdfbbc51d7cd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Where is the soccer Eurocup 2024 played?'}, 'id': 'call_gx8w', 'type': 'tool_call'}], usage_metadata={'input_tokens': 950, 'output_tokens': 57, 'total_tokens': 1007}),
 ToolMessage(content='[{"url": "https://e

In [13]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What is python?")]})

response["messages"]

[HumanMessage(content='What is python?', id='856c1e22-edf5-48ff-96b3-d56ffe3bad9b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x00f', 'function': {'arguments': '{"query":"what is python"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 943, 'total_tokens': 1013, 'completion_time': 0.2, 'prompt_time': 0.055337034, 'queue_time': 0.017800345000000002, 'total_time': 0.255337034}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-da0ba2f3-a6b0-4b78-ae2f-e26e43b22e06-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is python'}, 'id': 'call_x00f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 943, 'output_tokens': 70, 'total_tokens': 1013}),
 ToolMessage(content='[{"url": "https://www.pythontutorial.net/getting-started/what-is-python/", "content": "Learn wh

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and Where will it be the 2024 EUROCUP final Match, who was won the final match?")]}
):

    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g9na', 'function': {'arguments': '{"query":"2024 EUROCUP final match details"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 962, 'total_tokens': 1016, 'completion_time': 0.154285714, 'prompt_time': 0.046337232, 'queue_time': 0.030035046000000003, 'total_time': 0.200622946}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3d6688a6-5013-404d-957c-f7900267f395-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 EUROCUP final match details'}, 'id': 'call_g9na', 'type': 'tool_call'}], usage_metadata={'input_tokens': 962, 'output_tokens': 54, 'total_tokens': 1016})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024_Final", "content": "The UEFA

## Adding Memory

* Adding memory in LangGraph is very similar to what we did with LangChain.

In [16]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)
config = {"configurable": {"thread_id":"001"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer EUROCUP?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jyr5', 'function': {'arguments': '{"query":"2024 EUROCUP winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 950, 'total_tokens': 1002, 'completion_time': 0.156465091, 'prompt_time': 0.057655552, 'queue_time': 0.018585406999999998, 'total_time': 0.214120643}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-385dce3c-acca-4b10-9e98-c2e4577f0bbf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 EUROCUP winner'}, 'id': 'call_jyr5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 950, 'output_tokens': 52, 'total_tokens': 1002})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.sportingnews.com/us/soccer/news/who-won-euro-2024-spain-england-oyarzabal/19ab0b3bf6631

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of 2024 EUROCUP winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8s9q', 'function': {'arguments': '{"query":"2024 EUROCUP winner team top players"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 1379, 'total_tokens': 1427, 'completion_time': 0.137206354, 'prompt_time': 0.078399276, 'queue_time': 0.031992522999999995, 'total_time': 0.21560563}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7b7787dc-47bc-4a00-bb32-263a3c1e5aef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 EUROCUP winner team top players'}, 'id': 'call_8s9q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1379, 'output_tokens': 48, 'total_tokens': 1427})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.givemesport.com/ranking-best-players-euro-2024/", "c

In [21]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='We were talking about the Spain national soccer team, specifically their win in the 2024 European Championship (EUROCUP).', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1879, 'total_tokens': 1905, 'completion_time': 0.076634727, 'prompt_time': 0.115932354, 'queue_time': 0.11707601, 'total_time': 0.192567081}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bafce86-1df0-4d05-9dcf-eebd5a260de6-0', usage_metadata={'input_tokens': 1879, 'output_tokens': 26, 'total_tokens': 1905})]}}
----
